ArgoCD
======

![](https://argo-cd.readthedocs.io/en/stable/assets/argocd-ui.gif)

Quelle: [ArgoCD](https://argo-cd.readthedocs.io/)

- - -

[ArgoCD](https://argo-cd.readthedocs.io/) CD ist ein deklaratives GitOps-Continuous-Delivery-Tool für Kubernetes.

- - - 

Installation


In [ ]:
%%bash
kubectl create namespace argocd
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml
kubectl patch svc argocd-server -n argocd -p '{"spec": {"type": "LoadBalancer"}}'

In [ ]:
%%bash
echo "ArgoCD UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n argocd service/argocd-server -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "User     : admin"
echo "Password : $(kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d; echo)"

- - -

### Applikations Deklaration erstellen

`Create Application` drücken und folgende Werte abfüllen

- - -
* Application Name : ms-rest
* Project Name     : default
- - -
* Repository URL   : https://github.com/mc-b/misegr
* Path             : ewolff/ms-kubernetes
- - -
* Cluster URL      : https://kubernetes.default.svc
* Namespace        : ms-rest



- - -

### Applikation Bereitstellen (Deploy)

`SYNC` und dann `Synchronize` drücken.

Die Applikation steht dann unter nachfolgendem URL zur Verfügung:


In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')

- - -

### Asynchrone Applikation ms-kafka - Resource Application erstellen

Die zweite Resource erstellen wir mittels Deklaration statt über das UI.

Nach der Erstellung kann diese wie vorher mittels `SYNC` und dann `Synchronize` und aktiveren von `Auto Create Namespace` bereitgestellt (deploy) werden.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: ms-kafka
  namespace: argocd
spec:
  destination:
    namespace: ms-kafka
    server: https://kubernetes.default.svc
  project: default
  source:
    path: ewolff/ms-kafka
    repoURL: https://github.com/mc-b/misegr
    targetRevision: HEAD
%EOF%

echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-kafka apache -o=jsonpath='{ .spec.ports[0].nodePort }')